In [ ]:
# データ読み込み
import os
import numpy as np
import pandas as pd

# フォルダ名からコンペ名を取得
compe_name = os.listdir('/kaggle/input')[0]
print("Competition name: " + compe_name)

# データ読み込み
train = pd.read_csv('../input/%s/train.csv' % (compe_name))
test = pd.read_csv('../input/%s/test.csv' % (compe_name))
print("Train: ", train.shape)
print("Test: ", test.shape)

# 提出用ファイルの見本からIndexとObjectiveの列名を取得
sub_sample = pd.read_csv('../input/%s/sample_submission.csv' % (compe_name))
sub_idxcol = sub_sample.columns[0]
sub_objcol = sub_sample.columns[1]

print("Variables: ", train.columns)
print("Submission format: ")
print(sub_sample)
print("Index column name: " + sub_idxcol)
print("Objective column name: " + sub_objcol)

In [ ]:
# 学習データ・テストデータの準備
obj_var = 'Survived' # train.csvの目的変数の列名を指定する

xtrn = train.drop([obj_var], axis=1)
xtst = test.copy()
ytrn = train[obj_var]

# 目的変数（Survived）以外の変数一覧
#     True: 欠損値を含む変数
#     False: 欠損値を含まない変数
print(pd.merge(xtrn, xtst, 'outer').isnull().any())

In [ ]:
# 絶対に使用しない変数をリストアップ
drop_vars = ['PassengerId']

# 何らかの理由で使用しない変数をリストに追加
# drop_vars += ['Name', 'Sex', 'Age', 'Ticket', 'Fare', 'Cabin', 'Embarked']
drop_vars += ['Name', 'Ticket', 'Cabin']

# 欠損値を含む変数のリスト
cat_miss_vars = ['Embarked'] # カテゴリ変数
num_miss_vars = ['Age', 'Fare'] # 数値変数

# 使用する予定の変数（欠損値のない変数）
cat_vars = ['Sex'] # カテゴリ変数
num_vars = ['Pclass', 'SibSp', 'Parch'] # 数値変数

# 欠損値を含む変数を使用する変数リストに追加
cat_vars += cat_miss_vars
num_vars += num_miss_vars

In [ ]:
# 使用しない変数を削除
xtrn.drop(drop_vars, axis=1, inplace=True)
xtst.drop(drop_vars, axis=1, inplace=True)
print(xtrn)

In [ ]:
# 欠損値補完の処理

# カテゴリ変数の欠損箇所を「__NA__」という文字列で埋める
for c in cat_miss_vars:
    replace_cat = '__NA__'
    xtrn[c].fillna(replace_cat, inplace=True)
    xtst[c].fillna(replace_cat, inplace=True)

# 数値変数の欠損値を「学習データにおける平均値」で埋める
for c in num_miss_vars:
    replace_num = xtrn[c].mean()
    xtrn[c].fillna(replace_num, inplace=True)
    xtst[c].fillna(replace_num, inplace=True)
    
print(pd.merge(xtrn, xtst, 'outer').isnull().any())

In [ ]:
# 変数の組み合わせ
# # AgeとParchを組み合わせ
# xtrn["AgeParch"] = xtrn["Age"] * (xtrn["Parch"]+1)
# xtst["AgeParch"] = xtst["Age"] * (xtst["Parch"]+1)

xtrn["FarePclass"] = xtrn["Fare"] * xtrn["Pclass"]
xtst["FarePclass"] = xtst["Fare"] * xtst["Pclass"]

num_vars += ['FarePclass']
print(xtrn)

In [ ]:
# 変換前の数値変数
print(xtrn[num_vars])

from sklearn.preprocessing import MinMaxScaler

# # 学習データに基づいて複数列のYeo-Johnson変換を定義
# pt = PowerTransformer(method='box-cox')
# pt.fit(xtrn[num_vars])

#学習データに基づいて複数列のMin-Maxスケーリングを定義
pt = MinMaxScaler()
pt.fit(xtrn[num_vars])

# 変換後のデータで各列を置換
xtrn[num_vars] = pt.transform(xtrn[num_vars])
xtst[num_vars] = pt.transform(xtst[num_vars])

# 変換後の数値変数
print(xtrn[num_vars])

In [ ]:
# 変換前のカテゴリ変数
print(xtrn[cat_vars])

from sklearn.model_selection import KFold

# クロスバリデーションのfoldを定義
kf = KFold(n_splits=4, shuffle=True, random_state=71)

# 変数をループしてtarget encoding
for c in cat_vars:

    # targetを付加
    data_tmp = pd.DataFrame({c: xtrn[c], 'target': ytrn})
    target_mean = data_tmp.groupby(c)['target'].mean()
    
    # テストデータのカテゴリを置換
    xtst[c] = xtst[c].map(target_mean)
    
    # 変換後の値を格納する配列を準備
    tmp = np.repeat(np.nan, xtrn.shape[0])

    # 学習データからバリデーションデータを分ける
    for i, (tr_idx, va_idx) in enumerate(kf.split(xtrn)):
        # 学習データについて、各カテゴリにおける目的変数の平均を計算
        target_mean = data_tmp.iloc[tr_idx].groupby(c)['target'].mean()
        # バリデーションデータについて、変換後の値を一時配列に格納
        tmp[va_idx] = xtrn[c].iloc[va_idx].map(target_mean)

    # 変換後のデータで元の変数を置換
    xtrn[c] = tmp


# 変換後のカテゴリ変数
print(xtrn[cat_vars])

In [ ]:
# ---------------------------------
# 相関係数
# ---------------------------------
import scipy.stats as st

# 相関係数
corrs = []
for c in xtrn.columns:
    corr = np.corrcoef(xtrn[c], ytrn)[0, 1]
    corrs.append(corr)
corrs = np.array(corrs)

# ---------------------------------
# カイ二乗統計量
# ---------------------------------
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

# カイ二乗統計量
# x = MinMaxScaler().fit_transform(xtrn) # すでにMinMaxスケーリングを行っているので、省略
c2, _ = chi2(xtrn, ytrn)

# 重要度の上位を出力する（上位5個まで）
idx = np.argsort(c2)[::-1]
top_cols, top_importances = xtrn.columns.values[idx][7], corrs[idx][7]
print(top_cols, top_importances)

# 上位5つ以外の特徴量を削除
xtrn.drop(top_cols, axis=1, inplace=True)
xtst.drop(top_cols, axis=1, inplace=True)
print(xtrn)

In [ ]:
# PCA
from sklearn.decomposition import PCA

# データは標準化などのスケールを揃える前処理が行われているものとする
train_x_saved = xtrn.copy()
# 学習データに基づいてPCAによる変換を定義
pca = PCA() # PCA後に全主成分を取り出す場合
#pca = PCA(n_components=5) # PCA後に第1～5主成分のみ取り出す場合
pca.fit(xtrn)

# 変換の適用
xtrn = pca.transform(xtrn)
xtst = pca.transform(xtst)

print(train_x_saved[:1])
print(xtrn[:1])

In [ ]:
# # ---------------------------------
# # スタッキング
# # ----------------------------------
# from sklearn.metrics import log_loss
# from sklearn.model_selection import KFold

# # models.pyにModel1Xgb, Model1NN, Model2Linearを定義しているものとする
# # 各クラスは、fitで学習し、predictで予測値の確率を出力する

# from models import Model1Xgb, Model1NN, Model2Linear


# # 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
# def predict_cv(model, train_x, train_y, test_x):
#     preds = []
#     preds_test = []
#     va_idxes = []

#     kf = KFold(n_splits=4, shuffle=True, random_state=71)

#     # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
#     for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
#         tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
#         tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
#         model.fit(tr_x, tr_y, va_x, va_y)
#         pred = model.predict(va_x)
#         preds.append(pred)
#         pred_test = model.predict(test_x)
#         preds_test.append(pred_test)
#         va_idxes.append(va_idx)

#     # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
#     va_idxes = np.concatenate(va_idxes)
#     preds = np.concatenate(preds, axis=0)
#     order = np.argsort(va_idxes)
#     pred_train = preds[order]

#     # テストデータに対する予測値の平均をとる
#     preds_test = np.mean(preds_test, axis=0)

#     return pred_train, preds_test


# # 1層目のモデル
# # pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# # pred_test_1a, pred_test_1bは、テストデータの予測値
# model_1a = Model1Xgb()
# pred_train_1a, pred_test_1a = predict_cv(model_1a, xtrn, ytrn, xtst)

# model_1b = Model1NN()
# pred_train_1b, pred_test_1b = predict_cv(model_1b, train_x_nn, train_y, test_x_nn)

# # 1層目のモデルの評価
# print(f'logloss: {log_loss(train_y, pred_train_1a, eps=1e-7):.4f}')
# print(f'logloss: {log_loss(train_y, pred_train_1b, eps=1e-7):.4f}')

# # 予測値を特徴量としてデータフレームを作成
# train_x_2 = pd.DataFrame({'pred_1a': pred_train_1a, 'pred_1b': pred_train_1b})
# test_x_2 = pd.DataFrame({'pred_1a': pred_test_1a, 'pred_1b': pred_test_1b})

# # 2層目のモデル
# # pred_train_2は、2層目のモデルの学習データのクロスバリデーションでの予測値
# # pred_test_2は、2層目のモデルのテストデータの予測値
# model_2 = Model2Linear()
# pred_train_2, pred_test_2 = predict_cv(model_2, train_x_2, train_y, test_x_2)
# print(f'logloss: {log_loss(train_y, pred_train_2, eps=1e-7):.4f}')

In [ ]:
# XGBoostモデルの学習
from xgboost import XGBClassifier

# モデルの作成および学習データを与えての学習
model = XGBClassifier()
model.fit(xtrn, ytrn)

In [ ]:
# 欠損値を含む特徴量を使わない 0.761950
# NAと平均値で補完           0.792430
# PCA                      0.794210
# 重要度最下位の特徴量を削除　 0.789500
# 重要度最下位から2つを削除　 0.787500
# Age*Parch               0.791590
# Age+Parch               0.792660
# Age*Pclass              0.791270
# Fare*Pclass             0.793480
# Fare/Pclass             0.792920
# 学習済みモデルによる予測
from sklearn.metrics import accuracy_score

ptrn = model.predict_proba(xtrn)
ptst = model.predict_proba(xtst)
ltrn = np.argmax(ptrn, axis=1)
ltst = np.argmax(ptst, axis=1)

print(ptrn)
print(ltrn)
print(ytrn)

acctrn = accuracy_score(ytrn, ltrn)
print("XGBoost training accuracy: %f" % (acctrn))

In [ ]:
# 提出用ファイルを作成
csvname = 'submission_xgboost.csv'
print(csvname)
submission = pd.DataFrame({sub_idxcol: sub_sample[sub_idxcol], sub_objcol: ltst})
submission.to_csv(csvname, index=False)
print(submission)